In [1]:
import os
import mne
mne.set_log_level("error")

In [7]:
sub = "EL3002"
sub_dir = f"D:/Shaked_data/ISO/control_clean/{sub}"
filename = f"{sub}_196-head-ch_resample250_filtered_scored_bad-epochs"
bad_channels_path = f"{sub_dir}/{sub}_bad_channels.txt"

In [3]:
raw = mne.io.read_raw(f"{sub_dir}/{filename}.fif")

Look at the channels and mark bad ones!

In [9]:
raw.plot(duration=30)

In [4]:
def sort_electrode_names(electrode_list):
    return sorted(electrode_list, key=lambda x: int(x[1:]) if x.startswith('E') and x[1:].isdigit() else float('inf'))

In [10]:
original_bad_channels = sort_electrode_names([str(x) for x in raw.info['bads']])
bad_percentage = (len(original_bad_channels) / len(raw.ch_names)) * 100
print(f"Bad channels marked: {original_bad_channels}")
print(f"Count: {len(original_bad_channels)} ({bad_percentage:.2f}%)")

Bad channels marked: ['E9', 'E35', 'E45', 'E103', 'E216', 'E229']
Count: 6 (3.06%)


In [ ]:
with open(bad_channels_path, 'w') as f:
    for ch in original_bad_channels:
        f.write(f"{ch}\n")

In [8]:
# Load bad channels from file and set them as bad
if os.path.exists(bad_channels_path):
    with open(bad_channels_path, 'r') as f:
        loaded_bad_channels = [line.strip() for line in f if line.strip()]
    
    raw.info['bads'] = loaded_bad_channels
    print(f"Loaded and set {len(loaded_bad_channels)} bad channels from file:")
    print(f"Bad channels: {sort_electrode_names(loaded_bad_channels)}")
else:
    print(f"Bad channels file not found: {bad_channels_path}")

Loaded and set 6 bad channels from file:
Bad channels: ['E9', 'E35', 'E45', 'E103', 'E216', 'E229']


In [ ]:
raw.load_data()
raw.interpolate_bads()
print(f"Channels interpolated.")

In [17]:
raw.set_eeg_reference('average', projection=False, ch_type='eeg')
print(f"Re-referenced to average")

Re-referenced to average


In [13]:
raw.plot(duration=30)

In [18]:
output_path = f"{filename}_bad-channels_avg-ref"
raw.save(f"{dir}/{output_path}.fif")